<span style="color:blue">Copyright (c) 2014-2017 National Technology and Engineering
Solutions of Sandia, LLC. Under the terms of Contract DE-NA0003525
with National Technology and Engineering Solutions of Sandia, LLC,
the U.S. Government retains certain rights in this software.</span>    
    
<span style="color:blue">Redistribution and use in source and binary forms, with or without
modification, are permitted provided that the following conditions
are met:</span>    
    
&nbsp;&nbsp;&nbsp;&nbsp;<span style="color:green">1. Redistributions of source code must retain the above copyright
   notice, this list of conditions and the following disclaimer.</span>    
    
&nbsp;&nbsp;&nbsp;&nbsp;<span style="color:green">2. Redistributions in binary form must reproduce the above copyright
   notice, this list of conditions and the following disclaimer in the
   documentation and/or other materials provided with the distribution.</span>    
    
<span style="color:blue">THIS SOFTWARE IS PROVIDED BY THE COPYRIGHT HOLDERS AND CONTRIBUTORS
"AS IS" AND ANY EXPRESS OR IMPLIED WARRANTIES, INCLUDING, BUT NOT
LIMITED TO, THE IMPLIED WARRANTIES OF MERCHANTABILITY AND FITNESS FOR
A PARTICULAR PURPOSE ARE DISCLAIMED. IN NO EVENT SHALL THE COPYRIGHT
HOLDER OR CONTRIBUTORS BE LIABLE FOR ANY DIRECT, INDIRECT, INCIDENTAL,
SPECIAL, EXEMPLARY, OR CONSEQUENTIAL DAMAGES (INCLUDING, BUT NOT
LIMITED TO, PROCUREMENT OF SUBSTITUTE GOODS OR SERVICES; LOSS OF USE,
DATA, OR PROFITS; OR BUSINESS INTERRUPTION) HOWEVER CAUSED AND ON ANY
THEORY OF LIABILITY, WHETHER IN CONTRACT, STRICT LIABILITY, OR TORT
(INCLUDING NEGLIGENCE OR OTHERWISE) ARISING IN ANY WAY OUT OF THE USE
OF THIS SOFTWARE, EVEN IF ADVISED OF THE POSSIBILITY OF SUCH DAMAGE.</span>

Purpose: Arguments and code for drawing trajectories.    
    
Once we have a set of trajectories in memory, we can make decisions about how they should look on the screen. That includes ...    
    
&nbsp;&nbsp;&nbsp;&nbsp;* How should the line segments in the trajectory be colored?    
    
&nbsp;&nbsp;&nbsp;&nbsp;* What should the line width for each segment be?    
    
&nbsp;&nbsp;&nbsp;&nbsp;* Should there be a dot at the head of the trajectory? What size and color?    
     
&nbsp;&nbsp;&nbsp;&nbsp;* What layer (Z-order) should the trajectories live in?    
    
The convenience methods in this file will render a single group of trajectories. You can render several groups by instantiating several of them.

In [1]:
import matplotlib.colors

from tracktable.feature import annotations
from tracktable.render import paths

import numpy

In [2]:
def make_constant_colormap(color):
    colormap = matplotlib.colors.ListedColormap([ color ], 'dummy colormap')
    return colormap

<span style="color:blue">Render decorated trajectories onto a Basemap instance</span>    
    
Given a Basemap instance and an iterable containing trajectories, draw the trajectories onto the map with the specified appearance parameters. You can control the trajectory color, linewidth, z-order, and whether or not a dot is drawn at the head of each path.    
    
<span style="color:orange">Args:</span>    
&emsp;basemap: Basemap instance to draw into    
&emsp;trajectory_source: Iterable of Trajectory objects    
&emsp;trajectory_color_type: String, either 'scalar' or 'constant'    
&emsp;trajectory_color: Name of an annotation function if trajectory_color_type is 'scalar'. Name/hex string of a color if it's 'constant'    
&emsp;trajectory_colormap: Colormap to a map between scalars and colors if trajectory_color_type is 'scalar'    
&emsp;trajectory_zorder: Image layer for trajectory geometry -- higher layers occlude lower ones    
&emsp;decorate_trajectory_head: Whether or not to draw a dot at the head of each trajectory    
&emsp;trajectory_head_dot_size: Size (in points) for the dot at the head of each trajectory    
&emsp;trajectory_head_color: Name/hex string for the color of the trajectory dots    
&emsp;trajectory_linewidth: Trajectory linewidth in points    
&emsp;trajectory_initial_linewidth: If trajectory_linewidth is 'taper', lines will be this wide at the head of the trajectory    
&emsp;trajectory_final_linewidth: If trajectory_linewidth is 'taper', lines will be this wide at the tail of the trajectory    
&emsp;axes: Artists will be added to this Axes instance instead of the default    
    
<span style="color:orange">Raises:</span>    
&emsp;KeyError: trajectory_color_type is 'scalar' and the specified trajectory scalar generator was not found in tracktable.features.available_annotations()    
    
<span style="color:orange">Returns:</span>    
&emsp;A list of the artists added to the basemap    
    
<span style="color:blue">NOTE:</span> This function is an adapter between the trajectory_rendering argument group and the draw_traffic() function in the tracktable.render.paths module.

In [3]:
def render_trajectories(basemap,
                        trajectory_source,
                        trajectory_color_type="scalar",
                        trajectory_color="progress",
                        trajectory_colormap="gist_heat",
                        trajectory_zorder=10,
                        decorate_trajectory_head=False,
                        trajectory_head_dot_size=2,
                        trajectory_head_color="white",
                        trajectory_linewidth=0.5,
                        trajectory_initial_linewidth=0.5,
                        trajectory_final_linewidth=0.01,
                        scalar_min=0,
                        scalar_max=1,
                        axes=None):
    trajectories_to_render = None
    if trajectory_color_type == 'scalar':
        annotator = annotations.retrieve_feature_function(trajectory_color)
        def annotation_generator(traj_source):
            for trajectory in traj_source:
                yield(annotator(trajectory))

        trajectories_to_render = annotation_generator(trajectory_source)
        scalar_generator = annotations.retrieve_feature_accessor(trajectory_color)
        colormap = trajectory_colormap
    else:
        def dummy_scalar_retrieval(trajectory):
            scalar = numpy.zeros(len(trajectory))
            return scalar

        def dummy_generator(things):
            for thing in things:
                yield(thing)
        trajectories_to_render = dummy_generator(trajectory_source)
        scalar_generator = dummy_scalar_retrieval
        colormap = make_constant_colormap(trajectory_color)
    return render_annotated_trajectories(basemap,
                                         trajectories_to_render,
                                         trajectory_scalar_accessor=scalar_generator,
                                         trajectory_colormap=colormap,
                                         trajectory_zorder=trajectory_zorder,
                                         decorate_trajectory_head=decorate_trajectory_head,
                                         trajectory_head_dot_size=trajectory_head_dot_size,
                                         trajectory_head_color=trajectory_head_color,
                                         trajectory_linewidth=trajectory_linewidth,
                                         trajectory_initial_linewidth=trajectory_initial_linewidth,
                                         trajectory_final_linewidth=trajectory_final_linewidth,
                                         scalar_min=scalar_min,
                                         scalar_max=scalar_max,
                                         axes=axes)

In [4]:
def _dummy_accessor(trajectory):
    return numpy.zeros(len(trajectory))

<span style="color:blue">Render decorated trajectories onto a Basemap instance. The trajectory scalars must already be set.</span>    
    
Given a Basemap instance and an iterable containing trajectories, draw the trajectories onto the map with the specified appearance parameters. You can control the trajectory color, linewidth, z-order, and whether or not a dot is drawn at the head of each path.    
    
<span style="color:orange">Args:</span>    
&emsp;basemap: Basemap instance to draw into    
&emsp;trajectory_source: Iterable of Trajectory objects    
&emsp;trajectory_scalar_accessor: Return a list of scalars for a trajectory    
&emsp;trajectory_colormap: Colormap to a map between scalars and colors    
&emsp;trajectory_zorder: Image layer for trajectory geometry -- higher layers occlude lower ones    
&emsp;decorate_trajectory_head: Whether or not to draw a dot at the head of each trajectory    
&emsp;trajectory_head_dot_size: Size (in points) for the dot at the head of each trajectory    
&emsp;trajectory_head_color: Name/hex string for the color of the trajectory dots    
&emsp;trajectory_linewidth: Trajectory linewidth in points    
&emsp;trajectory_initial_linewidth: If trajectory_linewidth is 'taper', lines will be this wide at the head of the trajectory    
&emsp;trajectory_final_linewidth: If trajectory_linewidth is 'taper', lines will be this wide at the tail of the trajectory    
&emsp;axes: Artists will be added to this Axes instance instead of the default    
&emsp;scalar_min (float): Scalar value to map to bottom of color map    
&emsp;scalar_max( float): Scalar value to map to top of color map    
    
<span style="color:orange">Returns:</span>    
&emsp;A list of the artists added to the basemap    
    
<span style="color:blue">NOTE:</span> This function is an adapter between the trajectory_rendering argument group and the draw_traffic() function in the tracktable.render.paths module.

In [5]:
def render_annotated_trajectories(basemap,
                                  trajectory_source,
                                  trajectory_scalar_accessor=_dummy_accessor,
                                  trajectory_colormap="gist_heat",
                                  trajectory_zorder=10,
                                  decorate_trajectory_head=False,
                                  trajectory_head_dot_size=2,
                                  trajectory_head_color="white",
                                  trajectory_linewidth=0.5,
                                  trajectory_initial_linewidth=0.5,
                                  trajectory_final_linewidth=0.01,
                                  scalar_min=0,
                                  scalar_max=1,
                                  axes=None):
    if trajectory_linewidth == 'taper':
        linewidth_generator = _make_tapered_linewidth_generator(trajectory_initial_linewidth,
                                                                trajectory_final_linewidth)
    else:
        linewidth_generator = _make_constant_linewidth_generator(trajectory_linewidth)
    if decorate_trajectory_head:
        dot_size = trajectory_head_dot_size
        dot_color = trajectory_head_color
    else:
        dot_size = 0
        dot_color = 'white'
    return paths.draw_traffic(basemap,
                              trajectory_source,
                              color_map=trajectory_colormap,
                              trajectory_scalar_generator=trajectory_scalar_accessor,
                              trajectory_linewidth_generator=linewidth_generator,
                              zorder=trajectory_zorder,
                              color_scale=matplotlib.colors.Normalize(vmin=scalar_min, vmax=scalar_max),
                              dot_size=dot_size,
                              dot_color=dot_color,
                              axes=axes)


<span style="color:blue">Create a function that will make a tapered line width for a trajectory</span>    
    
In order to render tapered trajectories whose lines get thinner as they get older, we need to generate a scalar array with as many components as the trajectory segments. The first entry in this array (corresponding to the OLDEST point) should have the value 'final_linewidth'. The last entry (corresponding to the NEWEST point) should have the value 'initial_linewidth'.    
    
<span style="color:orange">Args:</span>    
&emsp;initial_linewidth: Width (in points) at the head of the trajectory    
&emsp;final_linewidth: Width (in points) at the tail of a trajectory    
    
<span style="color:orange">Returns:</span>    
&emsp;A function that takes in a trajectory as an argument and returns an array of linewidths    
    
<span style="color:blue">NOTE:</span> There might be an off-by-one error in here: We generate len(trajectory) scalars, but the geometry has len(trajectory)-1 segments. Check to see if draw_traffic in paths.py corrects for this.

In [6]:
def _make_tapered_linewidth_generator(initial_linewidth,
                                      final_linewidth):
    def linewidth_generator(trajectory):
        return numpy.linspace(final_linewidth, initial_linewidth, len(trajectory))
    return linewidth_generator

<span style="color:blue">Create a function that will make a constant line width for a trajectory</span>    
    
<span style="color:orange">Args:</span>    
&emsp;linewidth: Width (in points) along the trajectory    
    
<span style="color:orange">Returns:</span>    
&emsp;A function that takes in a trajectory as an argument and returns an array of linewidths    
    
<span style="color:blue">NOTE:</span> There might be an off-by-one error in here: We generate len(trajectory) scalars, but the geometry has len(trajectory)-1 segments. Check to see if draw_traffic in paths.py corrects for this.

In [7]:
def _make_constant_linewidth_generator(linewidth):
    def linewidth_generator(trajectory):
        scalars = numpy.zeros(len(trajectory))
        scalars += float(linewidth)
        return scalars
    return linewidth_generator